In [5]:
import pandas as pd
import numpy as np
import multiprocessing
from scipy import stats
import timeit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import scale
%matplotlib inline
import matplotlib.pyplot as plt
#import aux_functions
#from aux_functions import plot_with_err

# Import data

##Train

In [8]:
data = pd.read_csv('/content/top10_variables_train.csv')
y = data['good_bad_flag']
X = data.drop(["good_bad_flag"],axis=1)
X.head()

,avg_active_span_days,avg_interest,age,avg_paym_span_hrs,interest,avg_loanamount,max_loannumber,interest_interval_3000,loanamount,age_interval_36-40
0,33.000000,3000.000000,36.0,0.000000,3000.0,10000.000000,1.0,1,10000.0,1
1,3.000000,3000.000000,41.0,13.000000,3000.0,10000.000000,1.0,1,10000.0,0
2,23.333333,3133.333333,34.0,15.166667,3000.0,15000.000000,6.0,1,20000.0,0
3,25.333333,3000.000000,47.0,13.666667,4500.0,10000.000000,3.0,0,20000.0,0
4,14.000000,1559.090909,27.0,11.636364,4500.0,12727.272727,11.0,0,20000.0,0


Tipificamos

In [9]:
X_scale = pd.DataFrame(scale(X))
X_scale.columns = X.columns
X = X_scale
X.columns = X_scale.columns
X.head()

,avg_active_span_days,avg_interest,age,avg_paym_span_hrs,interest,avg_loanamount,max_loannumber,interest_interval_3000,loanamount,age_interval_36-40
0,1.276814,0.330176,-0.060081,-3.179329,-0.285638,-0.608786,-0.876227,1.260701,-0.733215,1.456264
1,-2.268915,0.330176,-0.059547,0.004068,-0.285638,-0.608786,-0.876227,1.260701,-0.733215,-0.686689
2,0.134301,0.491189,-0.060294,0.534634,-0.285638,0.439597,0.487956,1.260701,0.194972,-0.686689
3,0.370683,0.330176,-0.058906,0.167319,0.600455,-0.608786,-0.330554,-0.793209,0.194972,-0.686689
4,-0.968814,-1.409862,-0.061042,-0.329855,0.600455,-0.036941,1.852139,-0.793209,0.194972,-0.686689


In [10]:
#dimensión del conjunto de train
n_train = int(X.shape[0])

#selección del conjunto de train
X_train = X.iloc[:n_train]
y_train = y.iloc[:n_train]

## Validation y Test

In [11]:
data_t = pd.read_csv('/content/top10_variables_test.csv')
y_t = data_t['good_bad_flag']
X_t = data_t.drop(["good_bad_flag"],axis=1)
X_t.head()

,avg_active_span_days,avg_interest,age,avg_paym_span_hrs,interest,avg_loanamount,max_loannumber,interest_interval_3000,loanamount,age_interval_36-40
0,19.0,2700.0,39.0,12.8,4500.0,14000.0,5.0,0,20000.0,1
1,29.0,3000.0,42.0,13.0,3000.0,10000.0,2.0,1,10000.0,0
2,57.0,3000.0,45.0,7.0,3000.0,10000.0,1.0,1,10000.0,0
3,14.0,1500.0,40.0,18.0,3000.0,10000.0,1.0,1,10000.0,1
4,19.6,2700.0,37.0,9.0,4500.0,14000.0,5.0,0,20000.0,1


Tipificamos

In [12]:
X_t_scale = pd.DataFrame(scale(X_t))
X_t_scale.columns = X_t.columns
X_t = X_t_scale
X_t.columns = X_t_scale.columns
X_t.head()

,avg_active_span_days,avg_interest,age,avg_paym_span_hrs,interest,avg_loanamount,max_loannumber,interest_interval_3000,loanamount,age_interval_36-40
0,-0.289372,0.072892,-0.048716,-0.084439,0.678767,0.257389,0.293053,-0.755045,0.232470,1.468977
1,0.865412,0.425222,-0.048324,-0.036304,-0.233861,-0.596311,-0.566660,1.324424,-0.736634,-0.680746
2,4.098808,0.425222,-0.047932,-1.480364,-0.233861,-0.596311,-0.853231,1.324424,-0.736634,-0.680746
3,-0.866764,-1.336431,-0.048585,1.167079,-0.233861,-0.596311,-0.853231,1.324424,-0.736634,1.468977
4,-0.220085,0.072892,-0.048978,-0.999010,0.678767,0.257389,0.293053,-0.755045,0.232470,1.468977


Dividimos el dataset de test en validation y test, es decir, 50-50. Representarán el 10% del dataset total cada uno, ya que el train supone un 80% del total de los datos.

In [13]:
perc_values = [0.5, 0.5]

# dimensiones de los conjuntos de validation y test
n_val = int(X_t.shape[0] * perc_values[0])
n_test = int(X_t.shape[0] * perc_values[1])

# selección del conjunto de validación
X_val = X_t.iloc[:n_val]
y_val = y_t.iloc[:n_val]

# selección del conjunto de test
X_test = X_t.iloc[n_test:]
y_test = y_t.iloc[n_test:]

In [14]:
#Visualizamos el tamaño de los tres tipos de datasets

print('Train data size = ' + str(X_train.shape))
print('Train target size = ' + str(y_train.shape))
print('Validation data size = ' + str(X_val.shape))
print('Validation target size = ' + str(y_val.shape))
print('Test data size = ' + str(X_test.shape))
print('Test target size = ' + str(y_test.shape))

Train data size = (3361, 10)
Train target size = (3361,)
Validation data size = (420, 10)
Validation target size = (420,)
Test data size = (420, 10)
Test target size = (420,)


#Search Grid

In [15]:
#Hemos decidido probar los siguientes modelos:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [16]:
#Importamos la métrica que evaluará nuestros modelos
from sklearn.metrics import roc_auc_score as metric

## Parámetros generales

In [17]:
random_state = 42    # inicializa el random number generator interno
nthread = multiprocessing.cpu_count() - 1
scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)

In [18]:
#puntos recomendados por Cherkassky para la SVM

n = X_train.shape[0]
d = X_train.shape[1]
m = np.mean(y_train)
s = np.std(y_train)
C_cherk = np.max([np.abs(m + 3*s),np.abs(m - 3*s)])
gamma_cherk = np.power(0.2, 1/d)


## Definición del grid

In [19]:
# Regresion Logística
regularization_values = ['l1', 'l2', 'none']   #reduce overfitting
penalty_values = [1, 10, 100]   #penaliza por tener demasiadas variables

# SVM
C_values = [C_cherk, 5e-03, 4.5e-03, 4e-03]    #parámetro de regularización 
gamma_kernel_values = [gamma_cherk, 3.26e-09, 3.255e-09, 3.25e-09]    #influencia del train

# Random Forest
ntrees_values = [10, 100, 1000]
max_depth_values = [None, 6, 20];
min_samples_split_values = [2, 5, 20];
min_samples_leaf_values = [1, 5, 20];
max_features_values = [None, 1, 2];

# Xgboost
nrounds_values = [10, 100]
eta_values = [0.3, 0.99]
gamma_values = [0, 1]
max_depth_values = [6, 20]
min_child_weight_values = [1, 20]
subsample_values = [0.1, 1]
colsample_bytree_values = [0.1, 1]
num_parallel_tree_values = [1, 20]
lambda_values = [0, 1]
alpha_values = [0, 1]

In [20]:
params_values = [{'model': 'logistic regression',
                  'regularization': regularization_values,
                 'penalty': penalty_values},

                 {'model': 'svm',
                  'C': C_values,
                 'gamma_kernel': gamma_kernel_values},

                 {'model': 'random forest',
                  'n_trees': ntrees_values,
                 'max_depth': max_depth_values,
                 'min_samples_split': min_samples_split_values,
                 'min_samples_leaf': min_samples_leaf_values,
                 'max_features': max_features_values},

                 {'model': 'xgboost',
                  'nrounds': nrounds_values,
                  'eta': eta_values,
                 'gamma': gamma_values,
                 'max_depth': max_depth_values,
                 'min_child_weight': min_child_weight_values,
                 'subsample': subsample_values,
                 'colsample_bytree': colsample_bytree_values,
                 'num_parallel_tree': num_parallel_tree_values,
                 'lambda': lambda_values,
                 'alpha': alpha_values}]

In [21]:
total_iteraciones = 0
for params in params_values:
    if params['model'] == 'logistic regression':
        n = len(params['regularization'])*len(params['penalty'])
    elif params['model'] == 'svm':
        n = len(params['C'])*len(params['gamma_kernel'])
    elif params['model'] == 'random forest':
        n = len(params['n_trees'])*len(params['max_depth'])*len(params['min_samples_split'])*len(params['min_samples_leaf'])*len(params['max_features'])
    elif params['model'] == 'xgboost':
        n = len(params['nrounds'])*len(params['eta'])*len(params['gamma'])*len(params['max_depth'])*len(params['min_child_weight'])*len(params['subsample'])*len(params['colsample_bytree'])*len(params['num_parallel_tree'])*len(params['lambda'])*len(params['alpha'])
    total_iteraciones = total_iteraciones + n;
    print(str(n)+ ' iteraciones de ' + str(params['model']))
print(str(total_iteraciones)+ ' iteraciones en total')        

9 iteraciones de logistic regression
16 iteraciones de svm
162 iteraciones de random forest
1024 iteraciones de xgboost
1211 iteraciones en total


## Aplicamos modelos

In [22]:
grid_results = pd.DataFrame();
num_iter = 0
for params in params_values:
  
    #Regresion Logística
    if params['model'] == 'logistic regression':
        for regularization in params['regularization']:
            for penalty in params['penalty']:  
                start = timeit.default_timer()
                num_iter += 1
                print('Inicio de iteracion ' + str(num_iter) + '. Regularizacion = ' + str(regularization) + ', Lambda = '  + str(penalty) + '\n')
                
                #Entrenar modelo
                if regularization == 'l1':
                    model = LogisticRegression(penalty = regularization, solver = 'liblinear', C = penalty, random_state = random_state)
                else:
                    model = LogisticRegression(penalty = regularization,solver = 'lbfgs', C = penalty, random_state = random_state)
               
                model.fit(X_train, np.array(y_train))

                #Generar predicciones
                pred_train_p = model.predict_proba(X_train)
                pred_val_p = model.predict_proba(X_val)

                #Calcular métricas de evaluación
                auc_train = metric(y_train, pred_train_p[:, 1])
                auc_val = metric(y_val, pred_val_p[:, 1])
                time = timeit.default_timer() - start

                print('Fin de iteracion ' + str(num_iter) + '. Regularizacion = ' + str(regularization) + ', Lambda = '  + str(penalty) +
                      '. AUC train = '  + str(auc_train) + ' -  AUC val = '  + str(auc_val)  + ' -  time = '  + str(time)  + '\n')
                
                grid_results = grid_results.append(pd.DataFrame(data={'model':'Logistic Regression',
                                                                      'params': [{'regularization':[regularization], 'penalty':[penalty]}],
                                                                      'auc_train':[auc_train],
                                                                      'auc_val':[auc_val],
                                                                      'time':[time]},
                                                               columns=['model','params', 'auc_train', 'auc_val', 'time']), ignore_index=True)
                
    #SVM
    if params['model'] == 'svm':
        for C in params['C']:
            for gamma_kernel in params['gamma_kernel']:  
                start = timeit.default_timer()
                num_iter += 1
                print('Inicio de iteracion ' + str(num_iter) + '. C = ' + str(C) + ', gamma = '  + str(gamma_kernel) + '\n')
                
                #Entrenar modelo               
                model = SVC(C = C, gamma = gamma_kernel, probability = True, random_state = random_state)
                model.fit(X_train, np.array(y_train))

                #Generar predicciones
                pred_train_p = model.predict_proba(X_train)
                pred_val_p = model.predict_proba(X_val)

                #Calcular métricas de evaluación
                auc_train = metric(y_train, pred_train_p[:, 1])
                auc_val = metric(y_val, pred_val_p[:, 1])
                time = timeit.default_timer() - start

                print('Fin de iteracion ' + str(num_iter) + '. C = ' + str(C) + ', gamma = '  + str(gamma_kernel) + 
                      '. AUC train = '  + str(auc_train) + ' -  AUC val = '  + str(auc_val)  + ' -  time = '  + str(time)  + '\n')
                
                grid_results = grid_results.append(pd.DataFrame(data={'model':'SVM',
                                                                     'params': [{'C':[C], 'gamma_kernel':[gamma_kernel]}],
                                                                      'auc_train':[auc_train], 'auc_val':[auc_val], 'time':[time]},
                                                               columns=['model','params', 'auc_train', 'auc_val', 'time']), ignore_index=True)
                

    #Random Forest
    if params['model'] == 'random forest':
        for n_trees in params['n_trees']:
            for max_depth in params['max_depth']:
                for min_samples_split in params['min_samples_split']:  
                    for min_samples_leaf in params['min_samples_leaf']:  
                        for max_features in params['max_features']:  
                            start = timeit.default_timer()
                            num_iter += 1 
                            print('Inicio de iteracion ' + str(num_iter) + '. n_trees = ' + str(n_trees) + ', max_depth = ' + str(max_depth) + 
                                  ', min_samples_split = '  + str(min_samples_split) + ', min_samples_leaf = '  + str(min_samples_leaf) +
                                  ', max_features = '  + str(max_features) + '\n')

                            #Entrenar modelo               
                            model = RandomForestClassifier(n_estimators = n_trees, max_depth = max_depth,
                                                          min_samples_split = min_samples_split,
                                                          min_samples_leaf = min_samples_leaf,
                                                          max_features = max_features, random_state = random_state)

                            model.fit(X_train, np.array(y_train))

                            #Generar predicciones
                            pred_train_p = model.predict_proba(X_train)
                            pred_val_p = model.predict_proba(X_val)

                            #Calcular métricas de evaluación
                            auc_train = metric(y_train, pred_train_p[:, 1])
                            auc_val = metric(y_val, pred_val_p[:, 1])  
                            time = timeit.default_timer() - start

                            print('Fin de iteracion ' + str(num_iter) + '. n_trees = ' + str(n_trees) + ', max_depth = ' + str(max_depth) + 
                                  ', min_samples_split = '  + str(min_samples_split) + ', min_samples_leaf = '  + str(min_samples_leaf) +
                                  ', max_features = '  + str(max_features) + '. AUC train = '  + str(auc_train) +  ' -  AUC val = '  + str(auc_val)  + 
                                  ' -  time = '  + str(time)  + '\n')
                            
                            grid_results = grid_results.append(pd.DataFrame(data={'model':'random forest',
                                                                                  'params': [{'n_trees':[n_trees],'max_depth':[max_depth],
                                                                                              'min_samples_split':[min_samples_split],
                                                                                              'min_samples_leaf':[min_samples_leaf],
                                                                                              'max_features':[max_features]}],
                                                                                  'auc_train':[auc_train], 
                                                                                  'auc_val':[auc_val],
                                                                                  'time':[time]},
                                                                           columns=['model','params', 'auc_train', 'auc_val', 'time']), ignore_index=True)  
    #XGBOOST
    if params['model'] == 'xgboost':
         for nrounds in params['nrounds']:
            for eta in params['eta']:
                for gamma in params['gamma']:
                    for max_depth in params['max_depth']:
                        for min_child_weight in params['min_child_weight']:
                            for subsample in params['subsample']:
                                for colsample_bytree in params['colsample_bytree']:
                                    for num_parallel_tree in params['num_parallel_tree']:
                                        for lamda in params['lambda']:
                                            for alpha in params['alpha']:
                                                start = timeit.default_timer()
                                                num_iter += 1; 
                                                print('Inicio de iteracion ' + str(num_iter) + '. Parametro nrounds = ' + str(nrounds) + 
                                                      ', parametro eta = '  + str(eta) + ', parametro gamma = '  + str(gamma) +
                                                      ', parametro max_depth = '  + str(max_depth) + ', parametro min_child_weight = '  + str(min_child_weight) +
                                                      ', parametro subsample = '  + str(subsample) + ', parametro colsample_bytree = '  + str(colsample_bytree) +
                                                      ', parametro num_parallel_tree = '  + str(num_parallel_tree) + ', parametro lambda = '  + str(lamda) +
                                                      ', parametro alpha = '  + str(alpha) + '\n')
                                               
                                                #Entrenar modelo
                                                model = XGBClassifier(nthread = nthread, scale_pos_weight = scale_pos_weight,
                                                                      random_state = random_state, n_estimators = nrounds,
                                                                      learning_rate = eta, gamma = gamma,
                                                                      max_depth = max_depth, min_child_weight = min_child_weight ,
                                                                      subsample = subsample, colsample_bytree = colsample_bytree,
                                                                      num_parallel_tree = num_parallel_tree,
                                                                      reg_lambda = lamda, reg_alpha = alpha)
                                                model.fit(X_train, np.array(y_train))

                                                #Generar predicciones
                                                pred_train_p = model.predict_proba(X_train)
                                                pred_val_p = model.predict_proba(X_val)

                                                #Calcular métricas de evaluación
                                                auc_train = metric(y_train, pred_train_p[:, 1])
                                                auc_val = metric(y_val, pred_val_p[:, 1])    
                                                time = timeit.default_timer() - start

                                                print('Fin de iteracion ' + str(num_iter) + '. Parametro nrounds = ' + str(nrounds) + 
                                                      ', parametro eta = ' + str(eta) + ', parametro gamma = '  + str(gamma) +
                                                      ', parametro max_depth = '  + str(max_depth) + ', parametro min_child_weight = '  + str(min_child_weight) +
                                                      ', parametro subsample = '  + str(subsample) + ', parametro colsample_bytree = '  + str(colsample_bytree) +
                                                      ', parametro num_parallel_tree = '  + str(num_parallel_tree) + ', parametro lambda = '  + str(lamda) +
                                                      ', parametro alpha = '  + str(alpha) + '. AUC train = '  + str(auc_train) + 
                                                      ' -  AUC val = '  + str(auc_val)  + ' -  time = '  + str(time)  + '\n')
                                                grid_results = grid_results.append(pd.DataFrame(data={'model':'xgboost',
                                                                                                      'params': [{'nrounds':[nrounds], 'eta':[eta],
                                                                                                                  'gamma':[gamma], 'max_depth':[max_depth],
                                                                                                                  'min_child_weight':[min_child_weight],
                                                                                                                  'subsample':[subsample], 
                                                                                                                  'colsample_bytree':[colsample_bytree],
                                                                                                                  'num_parallel_tree':[num_parallel_tree],
                                                                                                                  'lamda':[lamda], 'alpha':[alpha]}],
                                                                                                      'auc_train':[auc_train], 'auc_val':[auc_val], 'time':[time]},
                                                                                                columns=['model', 'params', 'auc_train', 'auc_val', 'time']), ignore_index=True)

Inicio de iteracion 1. Regularizacion = l1, Lambda = 1

Fin de iteracion 1. Regularizacion = l1, Lambda = 1. AUC train = 0.6884141504671838 -  AUC val = 0.7395740905057675 -  time = 0.031027205000100366

Inicio de iteracion 2. Regularizacion = l1, Lambda = 10

Fin de iteracion 2. Regularizacion = l1, Lambda = 10. AUC train = 0.6884867777194011 -  AUC val = 0.7389402966155405 -  time = 0.035560394999947675

Inicio de iteracion 3. Regularizacion = l1, Lambda = 100

Fin de iteracion 3. Regularizacion = l1, Lambda = 100. AUC train = 0.6887427759062954 -  AUC val = 0.7390353656990747 -  time = 0.11368344999982583

Inicio de iteracion 4. Regularizacion = l2, Lambda = 1



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Fin de iteracion 4. Regularizacion = l2, Lambda = 1. AUC train = 0.6884095146851273 -  AUC val = 0.7388452275320065 -  time = 0.048347902999921644

Inicio de iteracion 5. Regularizacion = l2, Lambda = 10

Fin de iteracion 5. Regularizacion = l2, Lambda = 10. AUC train = 0.6885336506268607 -  AUC val = 0.738971986310052 -  time = 0.0419285159998708

Inicio de iteracion 6. Regularizacion = l2, Lambda = 100

Fin de iteracion 6. Regularizacion = l2, Lambda = 100. AUC train = 0.6886783900444005 -  AUC val = 0.7389402966155405 -  time = 0.044326684999987265

Inicio de iteracion 7. Regularizacion = none, Lambda = 1

Fin de iteracion 7. Regularizacion = none, Lambda = 1. AUC train = 0.6926543457881345 -  AUC val = 0.7269615920902521 -  time = 0.06362063100004889

Inicio de iteracion 8. Regularizacion = none, Lambda = 10



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1484: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio parameters"
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1484: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio parameters"


Fin de iteracion 8. Regularizacion = none, Lambda = 10. AUC train = 0.6926543457881345 -  AUC val = 0.7269615920902521 -  time = 0.05933897999989313

Inicio de iteracion 9. Regularizacion = none, Lambda = 100

Fin de iteracion 9. Regularizacion = none, Lambda = 100. AUC train = 0.6926543457881345 -  AUC val = 0.7269615920902521 -  time = 0.07448261200011075

Inicio de iteracion 10. C = 2.0232209027962953, gamma = 0.8513399225207846

Fin de iteracion 10. C = 2.0232209027962953, gamma = 0.8513399225207846. AUC train = 0.8640265887855281 -  AUC val = 0.6803143617695526 -  time = 3.4926762299999154

Inicio de iteracion 11. C = 2.0232209027962953, gamma = 3.26e-09

Fin de iteracion 11. C = 2.0232209027962953, gamma = 3.26e-09. AUC train = 0.636974997682109 -  AUC val = 0.6610787172011662 -  time = 1.9232720239999708

Inicio de iteracion 12. C = 2.0232209027962953, gamma = 3.255e-09

Fin de iteracion 12. C = 2.0232209027962953, gamma = 3.255e-09. AUC train = 0.6410539708048748 -  AUC val = 0

## Resultados

In [23]:
#por tiempo
grid_results.groupby(['model'], sort=False)['time'].mean().sort_values()

model
Logistic Regression    0.056924
xgboost                0.960268
random forest          1.962087
SVM                    2.155326
Name: time, dtype: float64

In [24]:
#por el AUC
grid_results.groupby(['model'], sort=False)['auc_val'].max().sort_values()

model
SVM                    0.696159
Logistic Regression    0.739574
random forest          0.758255
xgboost                0.759174
Name: auc_val, dtype: float64

In [25]:
#Elegimos la mejor combinación de modelo + hiperparámetros
best_params = grid_results.iloc[grid_results['auc_val'].idxmax()]

# Entrenamiento del modelo final

In [26]:
print('Train data size = ' + str(X_train.shape))
print('Train target size = ' + str(y_train.shape))
print('Validation data size = ' + str(X_val.shape))
print('Validation target size = ' + str(y_val.shape))

#Combinamos train y validación
X_train = pd.concat((X_train,X_val), axis = 0)
y_train = np.concatenate((y_train, y_val), axis = 0)

del X_val, y_val

print('Train data size = ' + str(X_train.shape))
print('Train target size = ' + str(y_train.shape))

Train data size = (3361, 10)
Train target size = (3361,)
Validation data size = (420, 10)
Validation target size = (420,)
Train data size = (3781, 10)
Train target size = (3781,)


In [27]:
#Regresión Logística
if best_params['model'] == 'logistic regression':       

    #Entrenar modelo
    if best_params['params']['regularization'] == 'l1':
        model = LogisticRegression(penalty = best_params['params']['regularization'][0], 
                                   solver = 'liblinear', C = best_params['params']['penalty'][0], random_state = random_state)
    else:
        model = LogisticRegression(penalty = best_params['params']['regularization'][0],
                                   solver = 'lbfgs', C = best_params['params']['penalty'][0], random_state = random_state)


#SVM
elif best_params['model'] == 'SVM':

    model = SVC(C = best_params['params']['C'][0], gamma = best_params['params']['gamma_kernel'][0], 
                probability = True, random_state = random_state)             


#Random Forest
elif best_params['model'] == 'random forest':
    model = RandomForestClassifier(n_estimators = int(best_params['params']['n_trees'][0]), max_depth = int(best_params['params']['max_depth'][0]),
                                                      min_samples_split = int(best_params['params']['min_samples_split'][0]),
                                                      min_samples_leaf = int(best_params['params']['min_samples_leaf'][0]),
                                                      max_features = int(best_params['params']['max_features'][0]), random_state = random_state)


# XGBOOST
elif best_params['model'] == 'xgboost':
    model = XGBClassifier(nthread = nthread, scale_pos_weight = scale_pos_weight, random_state = random_state,
                          n_estimators = int(best_params['params']['nrounds'][0]), learning_rate = best_params['params']['eta'][0], 
                          gamma = best_params['params']['gamma'][0], max_depth = int(best_params['params']['max_depth'][0]),
                          min_child_weight = best_params['params']['min_child_weight'][0], subsample = best_params['params']['subsample'][0],
                          colsample_bytree = best_params['params']['colsample_bytree'][0],
                          num_parallel_tree  = int(best_params['params']['num_parallel_tree'][0]),
                          reg_lambda = best_params['params']['lamda'][0], reg_alpha = best_params['params']['alpha'][0])

#Entrenar modelo
model.fit(X_train, np.array(y_train))

#Generar predicciones
pred_train_p = model.predict_proba(X_train)
pred_test_p = model.predict_proba(X_test)

#Calcular métricas de evaluación
auc_train = metric(y_train, pred_train_p[:, 1])
auc_test = metric(y_test, pred_test_p[:, 1]) 

results = pd.DataFrame()
results = results.append(pd.DataFrame(data={'model':best_params['model'],'auc_train':[auc_train],
                                            'auc_test':[auc_test]}, columns=['model',  'auc_train', 'auc_test']), ignore_index=True)

In [28]:
results

,model,auc_train,auc_test
0,xgboost,0.765915,0.65626
